In [1]:
#r "nuget: Microsoft.ML"
using Microsoft.ML;
using Microsoft.ML.Data;

Installed package Microsoft.ML version 1.5.5

This example comes from the ML.NET documentation: https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.extensionscatalog.indicatemissingvalues?view=ml-dotnet

In [2]:
class DataPoint
{
    [VectorType(3)]
    public float[] Features1 { get; set; }
    [VectorType(2)]
    public float[] Features2 { get; set; }
}

class SampleDataTransformed : DataPoint
{
    public bool[] MissingIndicator1 { get; set; }
    public bool[] MissingIndicator2 { get; set; }

}

In [3]:
var mlContext = new MLContext();

In [4]:
var samples = new List<DataPoint>()
{
    new DataPoint(){ Features1 = new float[3] {1, 1, 0},           Features2 = new float[2] {1, 1} },
    new DataPoint(){ Features1 = new float[3] {0, float.NaN, 1},   Features2 = new float[2] {float.NaN, 1} },
    new DataPoint(){ Features1 = new float[3] {-1, float.NaN, -3}, Features2 = new float[2] {1, float.PositiveInfinity} },
};

In [5]:
var data = mlContext.Data.LoadFromEnumerable(samples);

IndicateMissingValues is used to create a boolean containing 'true' where the value in the input column is missing. For floats and doubles, missing values are NaN. We can use an array of InputOutputColumnPair to apply the MissingValueIndicatorEstimator to multiple columns in one pass over the data.

In [6]:
var pipeline = mlContext.Transforms.IndicateMissingValues(new[] {
    new InputOutputColumnPair("MissingIndicator1", "Features1"),
    new InputOutputColumnPair("MissingIndicator2", "Features2")
});

In [7]:
var tansformer = pipeline.Fit(data);
var transformedData = tansformer.Transform(data);

In [8]:
mlContext.Data.CreateEnumerable<SampleDataTransformed>(transformedData, reuseRowObject: false)

index,MissingIndicator1,MissingIndicator2,Features1,Features2
0,"[ False, False, False ]","[ False, False ]","[ 1, 1, 0 ]","[ 1, 1 ]"
1,"[ False, True, False ]","[ True, False ]","[ 0, NaN, 1 ]","[ NaN, 1 ]"
2,"[ False, True, False ]","[ False, False ]","[ -1, NaN, -3 ]","[ 1, Infinity ]"
